In [6]:
import re
import numpy as np
import pandas as pd
import contractions
import inflect
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from bs4 import BeautifulSoup

import pandas as pd
import contractions
import re
from word2number import w2n
import nltk
#nltk.download()
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from tqdm import tqdm
from tqdm.auto import tqdm
tqdm.pandas()

from sklearn.preprocessing import MinMaxScaler
from math import log
import json
import matplotlib.pyplot as plt
import pickle

import os
import seaborn as sns
import scipy
import random
from datetime import datetime
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

In [7]:
!pip install word2number

In [9]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811628 sha256=488534420489bf0c147bb83f9d9efa5322faaf07e507b6c91cb4b9ab47cc3d2f
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [10]:
from surprise import SVD, KNNBasic, KNNWithMeans, KNNBaseline, NMF
from surprise import dump
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

from collections import defaultdict

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 언젠가 gpu 사용할때

# import torch

# '''
# if torch가 할당이 안되면요...
# 1) anaconda prompt 관리자권한 실행
# 2) conda activate janetorch(내가 쓰는 가상환경임)
# 2) conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia
# --> 내 버전 cuda11.7임....
# --> version : pytorch 2.0.1  pytorch-mutex 1.0  torchvision 0.15.2  py3.10_cuda 11.7  cudnn8.0  py310-cu117
# '''


# device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
# # GPU 사용 가능 -> True, GPU 사용 불가 -> False
# print(torch.cuda.is_available())

# print(torch.__version__)
# # # GPU 여러개를 지정하고 싶은 경우는 아래 형태로 사용
# # os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"

## 00. open data

In [12]:
path = '/content/drive/MyDrive/00.multi_emtions/03.models'

# 영화 정보가 담긴 데이터
# rating_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')

In [13]:
with open(os.path.join(path, 'movies_train.pkl'), 'rb') as f:
    train_set = pickle.load(f)
with open(os.path.join(path,'movies_test.pkl'), 'rb') as f:
    test_set = pickle.load(f)
with open(os.path.join(path,'movies_validation.pkl'), 'rb') as f:
    valid_set = pickle.load(f)

### 데이터 설명
- userID : 유저 아이디
- movieID : 영화 아이디
- rating : 유저가 영화에 남긴 평점
- compound : VADER 기준 sentiment score로 -1~1 범위(float)
- senti_score : NeMF를 위한 compound 기반 1~3의 sentiment score(compound -0.05 미만:1  /  -0.05~0.05:2  /  0.05 초과:3)
- NeMFRating : rating기준 1~3 정규화(rating 1,2 : 1  /  3 : 3  /  4,5 : 3)
- SBMFRating : SBMF를 위한 compound를 1~5점으로 정규화(min-max 정규화)
- anger~trust : 8 emotions based on NRCdictionary&LIWC
- negative, positive : sentiment score based on NRCdictionary&LIWC

In [14]:
len(train_set)

2250979

In [15]:
len(test_set)

643138

In [16]:
len(valid_set)

321569

In [24]:
seed_num = 42

# 01. MF(SVD, NMF)

In [ ]:
# !pip install -U pandas-profiling

In [ ]:
# mf_train = train_set[['userID','movieID','rating']]
# mf_test = test_set[['userID','movieID','rating']]
# mf_val = valid_set[['userID','movieID','rating']]

### 01-1. SVD gpt ver...

In [ ]:
combined_data = mf_train.append(mf_val)

In [ ]:
# Define the rating scale
rating_scale = (1, 5)  # Assuming ratings range from 1 to 5

# Create the Surprise dataset using the combined data and rating scale
reader = Reader(rating_scale=rating_scale)
data = Dataset.load_from_df(combined_data[['userID', 'movieID', 'rating']], reader)

# Build the full trainset
trainset = data.build_full_trainset()

In [ ]:
# Use the famous SVD algorithm.
algo = SVD()
algo.fit(trainset)


In [ ]:
# Predict ratings for the validation set
validation_data = Dataset.load_from_df(mf_val[['userID', 'movieID', 'rating']], reader)
validationset = validation_data.construct_testset(validation_data.raw_ratings)
predictions = algo.test(validationset)


RMSE: 0.6495


### 01-2. SVD version2

In [ ]:
# mf에 맞게 데이터지정
mf_train = train_set[['userID','movieID','rating']]
mf_test = test_set[['userID','movieID','rating']]
mf_val = valid_set[['userID','movieID','rating']]

In [ ]:
# version2 : train+valid as train  /  test for test

combined_data = mf_train.append(mf_val)

# rating 스케일링 해줘야하는듯
rating_scale = (1, 5)
reader = Reader(rating_scale=rating_scale)
data = Dataset.load_from_df(combined_data[['userID', 'movieID', 'rating']], reader)

# Build the full trainset
trainset = data.build_full_trainset()

# Define and train the matrix factorization algorithm
algo = SVD()
algo.fit(trainset)

# Predict ratings for the test set
test_data = Dataset.load_from_df(mf_test[['userID', 'movieID', 'rating']], reader)
testset = test_data.construct_testset(test_data.raw_ratings)
predictions = algo.test(testset)

In [ ]:
# RMSE with test data
predict = accuracy.rmse(predictions)
print(predict)

RMSE: 0.9724
0.9723782580007945


# 02. NMF, SVD - 랩실코드

In [ ]:
# data : train_set  /  valid_set  /  test_set

In [17]:
# trainset의 평점을 정규화
def norm(x):
    _max = x.max()
    _min = x.min()
    _denom = _max - _min
    return (x - _min) / _denom

In [18]:
train_data = train_set.append(valid_set)
# train_data['rating'] = norm(train_data['rating'])
train_data
# train_data / test_data 사용하면 됨.

,userID,movieID,rating,compound,senti_score,NeMFRating,SBMFRating,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive
1633870,143756,31800,5.0,0.5719,3,3,4,0.00,0.00,0.00,0.00,16.67,0.00,0.00,33.33,0.00,16.67
3115182,272600,13746,5.0,0.0000,2,3,3,0.00,11.11,0.00,11.11,0.00,0.00,0.00,11.11,0.00,0.00
2383131,208589,23849,3.0,-0.2023,1,2,3,10.53,5.26,5.26,5.26,0.00,5.26,5.26,5.26,10.53,10.53
2079042,182422,14736,4.0,0.9666,3,3,5,5.04,4.20,9.24,8.40,6.72,7.56,5.04,4.20,14.29,13.45
651398,56973,53386,5.0,0.9260,3,3,5,0.00,27.27,0.00,4.55,18.18,0.00,13.64,40.91,0.00,27.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318060,202972,31547,4.0,0.4404,3,3,4,0.00,3.57,0.00,0.00,3.57,0.00,3.57,7.14,0.00,3.57
2103736,184594,53967,5.0,0.9277,3,3,5,0.00,6.67,0.00,6.67,20.00,0.00,6.67,0.00,0.00,26.67
505244,43893,26640,4.0,0.9531,3,3,5,2.17,6.52,1.09,0.00,6.52,1.09,3.26,5.43,10.87,9.78
906648,79756,15221,5.0,0.6369,3,3,4,0.00,0.00,0.00,0.00,50.00,0.00,0.00,50.00,0.00,50.00


In [19]:
# 패키지 쓰려면 이거 설정해줘야함.
reader = Reader(rating_scale=(1, 5))
# train, test set 완성
trainset = Dataset.load_from_df(train_data[['userID', 'movieID', 'rating']], reader)
trainset = trainset.build_full_trainset()

testset = Dataset.load_from_df(test_set[['userID', 'movieID', 'rating']], reader)
testset = testset.construct_testset(testset.raw_ratings)


In [20]:
test_set

,userID,movieID,rating,compound,senti_score,NeMFRating,SBMFRating,anger,anticipation,disgust,fear,joy,sadness,surprise,trust,negative,positive
3158911,276374,48067,5.0,0.6249,3,3,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1704440,149783,541,5.0,0.6369,3,3,4,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,50.00
2971340,260113,47119,5.0,0.9600,3,3,5,0.00,12.00,0.00,0.00,16.00,0.00,16.00,16.00,0.00,28.00
1213057,106354,18996,1.0,0.9870,3,1,5,1.48,9.63,2.96,5.19,5.93,2.22,5.19,11.85,5.93,13.33
2806975,245581,28162,4.0,0.8625,3,3,5,0.00,23.08,0.00,0.00,7.69,0.00,7.69,7.69,0.00,23.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30437,2626,32298,5.0,0.9595,3,3,5,8.70,4.35,2.90,8.70,10.14,4.35,4.35,1.45,10.14,20.29
653889,57185,8722,5.0,0.9590,3,3,5,0.00,8.00,0.00,8.00,10.00,4.00,2.00,8.00,4.00,10.00
2224435,194822,28068,5.0,0.4215,3,3,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1501535,131790,49106,5.0,0.4404,3,3,4,0.00,100.00,0.00,0.00,100.00,0.00,100.00,100.00,0.00,100.00


## SVD

In [22]:
# number of neighborhood size
k = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [25]:
# 랩실코드

mae_svd = []
rmse_svd = []

for i in k:
    algo = SVD(n_factors = i, verbose = 0) # The number of iteration of the SGD procedure.
    np.random.seed(seed_num) # 결과 일정하게

    algo.fit(trainset)
    predictions_test = algo.test(testset)

    # prediction = 4 *((prediction - prediction.min())/(prediction.max() - prediction.min())) + 1

    error_mae = accuracy.mae(predictions_test, verbose = 0)
    error_rmse = accuracy.rmse(predictions_test, verbose = 0)

    mae_svd.append(error_mae)
    rmse_svd.append(error_rmse)

    print("K:", i, "MAE:",  error_mae,"RMSE",error_rmse)

K: 5 MAE: 0.6954269971150594 RMSE 0.9651160757269331
K: 10 MAE: 0.6948518631271329 RMSE 0.9648287509367067
K: 20 MAE: 0.6947674918425878 RMSE 0.9652455858255156
K: 30 MAE: 0.6956232510798915 RMSE 0.9671171171291648
K: 40 MAE: 0.6951373761746855 RMSE 0.9669670299145772
K: 50 MAE: 0.6962542732965378 RMSE 0.9680891798524659
K: 60 MAE: 0.697203235250543 RMSE 0.9694586665174542
K: 70 MAE: 0.6974930915046612 RMSE 0.9698493207727819
K: 80 MAE: 0.6981214385643816 RMSE 0.9707805911024535
K: 90 MAE: 0.6989038075737446 RMSE 0.9720204015341298
K: 100 MAE: 0.6990783201068497 RMSE 0.9721128236502431


In [26]:
SVD_result = {'k': k, 'mae_svd' : mae_svd, 'rmse_svd' : rmse_svd}

In [27]:
# 결과 데이터프레임으로 깔끔하게 보기
SVD_result_df = pd.DataFrame.from_dict(data=SVD_result, orient='columns')
SVD_result_df

,k,mae_svd,rmse_svd
0,5,0.695427,0.965116
1,10,0.694852,0.964829
2,20,0.694767,0.965246
3,30,0.695623,0.967117
4,40,0.695137,0.966967
5,50,0.696254,0.968089
6,60,0.697203,0.969459
7,70,0.697493,0.969849
8,80,0.698121,0.970781
9,90,0.698904,0.972020


## NMF

In [ ]:
# number of neighborhood size
k = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [ ]:
# NMF 랩실코드

mae_nmf = []
rmse_nmf = []

for i in k:
    algo = NMF(n_factors = i, verbose = False) # The number of iteration of the SGD procedure.
    np.random.seed(seed_num)

    algo.fit(trainset)
    predictions_test = algo.test(testset)

    error_mae = accuracy.mae(predictions_test, verbose = False)
    error_rmse = accuracy.rmse(predictions_test, verbose = False)

    mae_nmf.append(error_mae)
    rmse_nmf.append(error_rmse)

    print("K:", i, "MAE:",  error_mae,"RMSE",error_rmse)

K: 5 MAE: 1.12092662370317 RMSE 1.3029087913285646
K: 10 MAE: 0.8859472021606718 RMSE 1.1249342321945879
K: 20 MAE: 0.7050516036455786 RMSE 1.029458546061489
K: 30 MAE: 0.6512688820855519 RMSE 1.0208750486957938
K: 40 MAE: 0.6320716032204168 RMSE 1.0294667541738927
K: 50 MAE: 0.6267991285289032 RMSE 1.0428461897214643
K: 60 MAE: 0.6248357815423864 RMSE 1.0542570760560765
K: 70 MAE: 0.6261442008807708 RMSE 1.0652260939600373


In [ ]:
NMF_result = {'k': k, 'mae_nmf' : mae_nmf, 'rmse_nmf' : rmse_nmf}


In [ ]:
# 결과 데이터프레임으로 깔끔하게 보기
NMF_result_df = pd.DataFrame.from_dict(data=NMF_result, orient='columns')
NMF_result_df

In [ ]:
# 결과 저장해놓기
mf_result = pd.merge(NMF_result_df, SVD_result_df, on='k' )

In [ ]:
mf_result.to_csv('C:/Users/user/OneDrive/005. multi-emo_RS/03.models/00.result/01.MFresult.csv',index=False)

In [ ]:
## Normalize Version

In [ ]:
# trainset의 평점을 정규화
def norm(x):
    _max = x.max()
    _min = x.min()
    _denom = _max - _min
    return (x - _min) / _denom

In [ ]:
train_data = train_set.append(valid_set)
train_data['rating'] = norm(train_data['rating'])
train_data